 **Задание 1**
1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [10]:
rating = pd.read_csv('ratings.csv')
rating.head()

users=rating.groupby('userId').count()

users=users.loc[users.rating>100].head().reset_index().drop(['movieId','rating','timestamp'], axis=1)
users.head()
users_max=rating.groupby('userId').max().rename(columns={'timestamp':'time_max'}).reset_index().drop(['movieId','rating'], axis=1)
users_max.head()
users_min=rating.groupby('userId').min().rename(columns={'timestamp':'time_min'}).reset_index().drop(['movieId','rating'], axis=1)
users_min.head()
users_time=users.merge(users_max)
users_time=users_time.merge(users_min)
users_time['lifetime']=(users_time.time_max-users_time.time_min)
users_time

,userId,time_max,time_min,lifetime
0,4,949982274,949778714,203560
1,8,1154474527,1154389340,85187
2,15,1469330735,997937239,471393496
3,17,1127476640,1127468587,8053
4,19,855195373,855190091,5282


**Задание  2**
2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [68]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [69]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [70]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [71]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [75]:
logistic=rzd.merge(auto, how='outer').merge(air, how='outer')
logistic

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [76]:
logistic=rzd.merge(auto, how='outer').merge(air, how='outer').merge(client_base,  how='outer')
logistic

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


**Задание 3**
3. В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Зная координаты пользователей в момент совершения покупок можно провести анализ популярности видов товаров (покупок) в той или иной местности.Для этого, кроме координат, необходимо знать дату и время совершения покупки (время регистрации пользователя в точке привязки координат). Так же, для определенных товаров можно проанализировать спрос на группы товаров относительно времени суток. Данные о покупках и данные о местоположении пользователя можно связать через UserID. Как следует из условий задачи этот параметр присутствует в обеих таблицах.